In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("F://machine learning//mlops//end to end machine learning pipeline//MLOPs_workflow//data//processed//feature_store_data//training_data.csv")

In [4]:
data.head(2)

,price,id,event_timestamp,room_type,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,minimum_nights,...,calculated_host_listings_count,availability_365,longitude,latitude,last_review,apartment,bedroom,cozy,private,room
0,149,2539,1890-06-18 14:06:42.611608+00:00,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,6.0,365.0,-73.97237,40.64749,262.0,0.0,0.0,0.0,0.0,0.0
1,225,2595,1890-06-19 14:06:42.611608+00:00,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,2.0,355.0,-73.98377,40.75362,48.0,0.0,0.0,0.0,0.0,0.0


In [8]:
data.shape

(48895, 22)

In [16]:
data.index[48894]

48894